<a href="https://colab.research.google.com/github/JayNguyen-123/TASK_AUTOMATION_PYTHON/blob/main/Task_Automation_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Python function to organize files

In [ ]:
import os
import shutil
from pathlib import Path
from datetime import datetime

def organize_files(directory):
  """Automatically organize files into folders by type"""
  # Define file categories
  file_types= {
      'Images': ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.svg'],
      'Documents': ['.pdf', '.doc', '.docx', '.txt', '.xlsx', '.pptx'],
      'Videos': ['.mp4', '.avi', '.mov', '.mkv'],
      'Audio': ['.mp3', '.wav', '.rar', '.7z', '.tar'],
      'Code': ['.py', '.js', '.html', '.css', '.java']
  }

  directory_path = Path(directory)

  for file in directory_path.iterdir():
    if file.is_file():
      file_ext = file.suffix.lower()

      # Find the right category
      moved = False
      for category, extensions in file_types.items():
        if file_ext in extensions:
          # Create category folder if it doesn't exist
          category_path = directory_path / category
          category_path.mkdir(exist_ok=True)

          # Move the file
          try:
            shutil.move(str(file), str(category_path / file.name))
            print(f"Moved: {file.name} -> {category}/")
            moved = True
            break
          except Exception as e:
            print(f"Error moving {file.name}: {e}")
      if not moved and file_ext:
        # Create "Others" folder for unrecognized types
        others_path = directory_path / "Others"
        others_path.mkdir(exist_ok=True)
        try:
          shutil.move(str(file), str(others_path / file.name))
          print(f"Moved: {file.name} -> Others/")
        except Exception as e:
          print(f"Error moving {file.name}: {e}")

# Use case
organize_files('C:/Users/User/Downloads')



### Excel Report Generator



In [ ]:
import pandas as pd
from datetime import datetime
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.chart import BarChart, Reference

def create_weekly_report(data_file, output_name="Weekly Report"):
  """ Generate a formatted excel report with charts."""

  # Read data (CSV, Exel, etc.)
  df = pd.read_csv(data_file)

  # Create a new Excel file
  timestamp = datetime.now().strftime("%Y-%m-%d")
  output_file = f"{output_name}_{timestamp}.xlsx"

  with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Write summary statistics
    summary = df.describe()
    summary.to_excel(writer, sheet_name='Summary')

    # Write full data
    df.to_excel(writer, sheet_name='Data', index=False)

    # Acccess the workbook to add formatting
    workbook = writer.book
    summary_sheet = workbook['Summary']
    data_sheet = workbook['Data']

    # Format headers
    header_fill = PatternFill(start_color='366092', end_color='366092', fill_type='solid')
    header_font = Font(bold=True, color='FFFFFF')

    for cell in data_sheet[1]:
      cell.fill = header_fill
      cell.font = header_font
      cell.alignment = Alignment(horizontal='center')

    # Auto-adjust widths
    for column in data_sheet.columns:
      max_length = 0
      column_letter = column[0].column_letter
      for cell in column:
        try:
          if len(str(cell.value)) > max_length:
            max_length = len(cell.value)
        except:
          pass

      adjusted_width = min(max_length + 2, 50)
      data_sheet.column_dimensions[column_letter].width = adjusted_width

    # Add a chart if we have numeric data
    if len(df.select_dtypes(include=['number']).columns) > 0:
      chart_sheet = workbook.create_sheet('Charts')
      chart = BarChart()
      chart.title = "Data Overview"
      chart.stype = 10

      # Use first 10 rows for the chart
      data = Reference(data_sheet, min_col=2, min_row=1, max_row=min(11, len(df)+1), max_col=len(df.columns))
      categories =Reference(data_sheet, min_col=1, min_row=2, max_row=min(11, len(df)+1))

      chart.add_data(data, titles_from_data=True)
      chart.set_categories(categories)
      chart_sheet.add_chart(chart, "A1")

  print(f"Report created: {output_file}")
  return output_file

# Use case
create_weekly_report("sale_data.csv", "Weekly_Report")



### Python function to handles email backlogs


In [ ]:
import imaplib
import email
from email.header import decode_header
import re
from datetime import datetime, timedelta
from getpass import getpass

def process_emails(email_address, password, days_back=1):
  """
  Automatically sort and categorize emails
  Note: Use an app-specific password, not your main password
  """

  # Connect to email server (Gmail example)
  mail = imaplib.IMAP4_SSL("imap.gmail.com")
  mail.login(email_address, password)
  mail.select("inbox")

  # Search for recent emails
  date = (datetime.now() - timedelta(days=days_back)).strftime("%d-%b-%Y")
  status, messages = email.search(None, f'SINCE {date}')
  email_ids = messages[0].split()

  # Categories for sorting
  categories = {
      'Urgen': ['urgent', 'asap', 'immediately', 'critical'],
      'Meetings': ['meeting', 'schedule', 'calendar', 'zoom', 'call'],
      'Invoices': ['invoice', 'payment', 'receipt', 'billing'],
      'Report': ['report', 'weekly', 'monthly', 'could you', 'help']
  }

  sorted_emails = {cat: [] for cat in categories}
  sorted_emails['Other'] = []

  print(f"\nProcessing {len(email_ids)} emails...\n")

  for email_id in email_ids[-20]:
    # Process last 20 emails
    status, msg_data = email.fetch(email_id, "RFC822")

    for response_part in msg_data:
      if isinstance(response_part, tuple):
        msg = email.message_from_bytes(response_part[1])

        # Get subject
        subject = decode_header(msg["Subject"])[0][0]
        if isinstance(subject, bytes):
          subject = subject.decode()

        # Get sender
        from_ = msg.get("From")

        # Get email body
        body = ""
        if msg.is_multipart():
          for part in msg.walk():
            if part.get_content_type() == "text/plain":
              body = part.get_payload(decode=True).decode()
              break
        else:
          body = msg.get_payload(decode=True).decode()

        # Categorize
        categorized = False
        search_text = f"{subject} {body}".lower()

        for category, keywords in categories.items():
          if any(keyword in search_text for keyword in keywords):
            sorted_emails[category].append({
                "from": from_,
                "subject": subject,
                "preview": body[:100]
            })
            categorized = True
            break

        if not categorized:
          sorted_emails['Other'].append({
              "from": from_,
              "subject": subject,
              "preview": body[:100]
          })

  # Print results
  for category, emails in sorted_emails.items():
    if emails:
      print(f"\n {category} ({len(email)} emails):")
      for e in emails[:3]:
        # Show first 3
        print(f"  * From {e['from']}")
        print(f"    Subject: {e['subject']}")
        print()

  mail.close()
  mail.logout()

  print("Email processing complete!")
  return sorted_emails

# Use case
# Important: Never use your main password. Create an "App Password" in email settings
email_address = input("Enter your email address: ")
password = getpass("Enter your password: ")
process_emails(email_address, password, days_back=1)

